# **Movies Dataset Data Wrangling**

## **Collaborators** 
- Ashna Sood 
- Urmi Suresh
- Tae Kim 
- Xianglong Wang

## **Imports** 

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import os 
import pickle
import math

import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')

import patsy
import statsmodels.api as sm
import scipy.stats as stats

from sklearn.metrics import make_scorer, accuracy_score, plot_confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

from nltk.stem.snowball import SnowballStemmer

from sklearn import metrics

## **Data Cleaning**

In [2]:
# read in movies metadata 
movies_df = pd.read_csv('Movies Data/movies_metadata.csv')

# drop unecessary metadata
movies_df = movies_df.drop(columns=["adult", "budget", "homepage", "original_title", "overview", "poster_path", "status", "tagline", "video"])


# change column names
movies_df = movies_df.rename(columns={"belongs_to_collection":"Collection", 
                                      "genres":"Genres", 
                                      "id":"ID", 
                                      "imdb_id":"IMDB ID", 
                                      "original_language":"Language", 
                                      "popularity":"Popularity Rating", 
                                      "production_companies":"Production Companies", 
                                      "production_countries":"Production Countries", 
                                      "release_date":"Release Date", 
                                      "revenue":"Revenue", "runtime":"Runtime", 
                                      "spoken_languages":"Spoken Languages", 
                                      "title":"Title", 
                                      "vote_average":"Vote Average", 
                                      "vote_count":"Vote Count"})

# remove duplicate movies present in the df keeping the movie with a higher vote count
movies_df = movies_df.sort_values('Vote Count').drop_duplicates('Title', keep='last')

# reorder columns 
movies_df = movies_df[["ID", "IMDB ID", "Title", "Collection", "Genres", "Language", "Spoken Languages", "Release Date", 
                       "Runtime", "Revenue", "Production Companies", "Production Countries", 
                       "Popularity Rating", "Vote Count", "Vote Average"]]

movies_df

,ID,IMDB ID,Title,Collection,Genres,Language,Spoken Languages,Release Date,Runtime,Revenue,Production Companies,Production Countries,Popularity Rating,Vote Count,Vote Average
45465,461257,tt6980792,Queerama,NaN,[],en,"[{'iso_639_1': 'en', 'name': 'English'}]",2017-06-09,75.0,0.000000e+00,[],"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",0.163015,0.0,0.0
23677,92323,tt0081758,Willie and Phil,NaN,[],en,[],1980-08-15,115.0,0.000000e+00,[],[],0.3265,0.0,0.0
23671,114838,tt0029949,Brother Rat,NaN,"[{'id': 35, 'name': 'Comedy'}]",en,"[{'iso_639_1': 'en', 'name': 'English'}]",1938-10-29,87.0,0.000000e+00,"[{'name': 'Warner Bros.', 'id': 6194}]","[{'iso_3166_1': 'US', 'name': 'United States o...",0.174691,0.0,0.0
23663,264723,tt0070580,Le pélican,NaN,[],en,[],1974-02-06,83.0,0.000000e+00,[],[],0.000115,0.0,0.0
23648,88061,tt0055459,"So Evil, So Young",NaN,"[{'id': 18, 'name': 'Drama'}]",en,"[{'iso_639_1': 'en', 'name': 'English'}]",1963-01-01,77.0,0.000000e+00,[],[],0.001662,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17818,24428,tt0848228,The Avengers,"{'id': 86311, 'name': 'The Avengers Collection...","[{'id': 878, 'name': 'Science Fiction'}, {'id'...",en,"[{'iso_639_1': 'en', 'name': 'English'}]",2012-04-25,143.0,1.519558e+09,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",89.8876,12000.0,7.4
14551,19995,tt0499549,Avatar,"{'id': 87096, 'name': 'Avatar Collection', 'po...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",2009-12-10,162.0,2.787965e+09,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",185.071,12114.0,7.2
12481,155,tt0468569,The Dark Knight,"{'id': 263, 'name': 'The Dark Knight Collectio...","[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",2008-07-16,152.0,1.004558e+09,"[{'name': 'DC Comics', 'id': 429}, {'name': 'L...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",123.167,12269.0,8.3
15480,27205,tt1375666,Inception,NaN,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",en,"[{'iso_639_1': 'en', 'name': 'English'}]",2010-07-14,148.0,8.255328e+08,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",29.1081,14075.0,8.1


In [3]:
# read in movie keywords
keywords_df = pd.read_csv('Movies Data/keywords.csv')

# rename columns
keywords_df = keywords_df.rename(columns={"id": "ID", 
                                          "keywords": "Keywords"})

In [4]:
# read in movie credits 
credits_df = pd.read_csv('Movies Data/credits.csv')

# rename columns
credits_df = credits_df.rename(columns={"cast": "Cast", 
                                        "crew": "Crew",
                                        "id": "ID"})

In [5]:
# merge tables
movies_df['ID'] = movies_df['ID'].astype(str)
keywords_df['ID'] = keywords_df['ID'].astype(str)
credits_df['ID'] = credits_df['ID'].astype(str)

movies_df = movies_df.merge(keywords_df, on="ID", how="left").merge(credits_df, on="ID", how="left")
movies_df.head(1)

,ID,IMDB ID,Title,Collection,Genres,Language,Spoken Languages,Release Date,Runtime,Revenue,Production Companies,Production Countries,Popularity Rating,Vote Count,Vote Average,Keywords,Cast,Crew
0,461257,tt6980792,Queerama,NaN,[],en,"[{'iso_639_1': 'en', 'name': 'English'}]",2017-06-09,75.0,0.0,[],"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",0.163015,0.0,0.0,[],[],"[{'credit_id': '593e676c92514105b702e68e', 'de..."


In [6]:
# cast types
movies_df['Release Date'] = pd.to_datetime(movies_df['Release Date'], errors='coerce')
movies_df['Popularity Rating'] = pd.to_numeric(movies_df['Popularity Rating'], errors='coerce')
movies_df['Vote Count'] = movies_df['Vote Count'].astype(np.int64, errors='ignore')

# replace unreasonable values with NaN
movies_df['Revenue'] = movies_df['Revenue'].replace(0, np.nan)
movies_df['Popularity Rating'] = movies_df['Popularity Rating'].replace(0, np.nan)
movies_df['Vote Count'] = movies_df['Vote Count'].replace(0, np.nan)
movies_df['Vote Average'] = movies_df['Vote Average'].replace(0, np.nan)

In [7]:
# method to extract the director, screenplay writer, and producer from each movie
def extract_crew(crew):
    director, writer, producer = None, None, None
    for each in crew:

        if director and writer and producer:
            return [director, writer, producer]

        try:
            if (not director) and each['department']=='Directing' and each['job']=='Director':
                director = each['name']
                continue
                
        except:
          ...

        try:
            if (not writer) and each['department']=='Writing' and each['job']=='Screenplay' or each['job']=='Writer':
                writer = each['name']
                continue
        except:
            ...

        try:
            if (not producer) and each['department']=='Production' and (each['job']=='Producer' or each['job']=='Executive Producer'):
                producer = each['name']
                continue
        except:
            ...

    return [director, writer, producer]

In [8]:
# convert the Collections string literal into a dict and extract the name of the franchise 
movies_df["Collection"] = movies_df['Collection'].fillna('[]').apply(ast.literal_eval).apply(lambda x: x["name"] if isinstance(x, dict) else np.nan)

# extract genres out of string literal
movies_df['Genres'] = movies_df['Genres'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else np.nan)

# extract spoken languages out of string literal
movies_df['Spoken Languages'] = movies_df['Spoken Languages'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['iso_639_1'] for i in x] if isinstance(x, list) else np.nan)

# extract Production Company out of string literal
movies_df['Production Companies'] = movies_df['Production Companies'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else np.nan)

# extract Production Countries out of string literal
movies_df['Production Countries'] = movies_df['Production Countries'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else np.nan)

# extract the keywords for each movie
movies_df["Keywords"] = movies_df["Keywords"].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else np.nan)

# extract the names of at most the top 5 actors in each movie
movies_df["Cast"] = movies_df["Cast"].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else np.nan)
movies_df["Cast"] = movies_df["Cast"].apply(lambda x: x[:5] if len(x) > 5 else x)

# conver the Crew string literal into dict and extract the director, screenplay writer, and producer of each movie 
movies_df["Crew"] = movies_df["Crew"].fillna('[]').apply(ast.literal_eval)
movies_df['Director'], movies_df['Writer'], movies_df['Producer'] = [*zip(*np.array(movies_df['Crew'].apply(extract_crew)))]
movies_df = movies_df.drop('Crew', axis=1)

movies_df = movies_df.fillna(value=np.nan)

movies_df

,ID,IMDB ID,Title,Collection,Genres,Language,Spoken Languages,Release Date,Runtime,Revenue,Production Companies,Production Countries,Popularity Rating,Vote Count,Vote Average,Keywords,Cast,Director,Writer,Producer
0,461257,tt6980792,Queerama,NaN,[],en,[en],2017-06-09,75.0,NaN,[],[United Kingdom],0.163015,NaN,NaN,[],[],Daisy Asquith,NaN,NaN
1,92323,tt0081758,Willie and Phil,NaN,[],en,[],1980-08-15,115.0,NaN,[],[],0.326500,NaN,NaN,[],"[Michael Ontkean, Ray Sharkey, Margot Kidder]",Paul Mazursky,Paul Mazursky,NaN
2,114838,tt0029949,Brother Rat,NaN,[Comedy],en,[en],1938-10-29,87.0,NaN,[Warner Bros.],[United States of America],0.174691,NaN,NaN,"[based on play or musical, virginia military i...","[Ronald Reagan, Jane Wyman, Priscilla Lane, Wa...",William Keighley,Jerry Wald,NaN
3,264723,tt0070580,Le pélican,NaN,[],en,[],1974-02-06,83.0,NaN,[],[],0.000115,NaN,NaN,[],[],Gérard Blain,NaN,NaN
4,88061,tt0055459,"So Evil, So Young",NaN,[Drama],en,[en],1963-01-01,77.0,NaN,[],[],0.001662,NaN,NaN,"[prison, women's prison]","[Jill Ireland, Ellen Pollock, Joan Haythorne, ...",Godfrey Grayson,Mark Grantham,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43246,24428,tt0848228,The Avengers,The Avengers Collection,"[Science Fiction, Action, Adventure]",en,[en],2012-04-25,143.0,1.519558e+09,"[Paramount Pictures, Marvel Studios]",[United States of America],89.887648,12000.0,7.4,"[new york, shield, marvel comic, superhero, ba...","[Robert Downey Jr., Chris Evans, Mark Ruffalo,...",Joss Whedon,Joss Whedon,Stan Lee
43247,19995,tt0499549,Avatar,Avatar Collection,"[Action, Adventure, Fantasy, Science Fiction]",en,"[en, es]",2009-12-10,162.0,2.787965e+09,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",185.070892,12114.0,7.2,"[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron,James Cameron,James Cameron
43248,155,tt0468569,The Dark Knight,The Dark Knight Collection,"[Drama, Action, Crime, Thriller]",en,"[en, zh]",2008-07-16,152.0,1.004558e+09,"[DC Comics, Legendary Pictures, Warner Bros., ...","[United Kingdom, United States of America]",123.167259,12269.0,8.3,"[dc comics, crime fighter, secret identity, sc...","[Christian Bale, Michael Caine, Heath Ledger, ...",Christopher Nolan,Christopher Nolan,Charles Roven
43249,27205,tt1375666,Inception,NaN,"[Action, Thriller, Science Fiction, Mystery, A...",en,[en],2010-07-14,148.0,8.255328e+08,"[Legendary Pictures, Warner Bros., Syncopy]","[United Kingdom, United States of America]",29.108149,14075.0,8.1,"[loss of lover, dream, kidnapping, sleep, subc...","[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle...",Christopher Nolan,Christopher Nolan,Christopher Nolan


## **Data Wrangling for Content Based Recommender System**

In [15]:
movies_df

,ID,IMDB ID,Title,Collection,Genres,Language,Spoken Languages,Release Date,Runtime,Revenue,Production Companies,Production Countries,Popularity Rating,Vote Count,Vote Average,Keywords,Cast,Director,Writer,Producer
0,461257,tt6980792,Queerama,NaN,[],en,[en],2017-06-09,75.0,NaN,[],[United Kingdom],0.163015,NaN,NaN,[],[],Daisy Asquith,NaN,NaN
1,92323,tt0081758,Willie and Phil,NaN,[],en,[],1980-08-15,115.0,NaN,[],[],0.326500,NaN,NaN,[],"[Michael Ontkean, Ray Sharkey, Margot Kidder]",Paul Mazursky,Paul Mazursky,NaN
2,114838,tt0029949,Brother Rat,NaN,[Comedy],en,[en],1938-10-29,87.0,NaN,[Warner Bros.],[United States of America],0.174691,NaN,NaN,"[based on play or musical, virginia military i...","[Ronald Reagan, Jane Wyman, Priscilla Lane, Wa...",William Keighley,Jerry Wald,NaN
3,264723,tt0070580,Le pélican,NaN,[],en,[],1974-02-06,83.0,NaN,[],[],0.000115,NaN,NaN,[],[],Gérard Blain,NaN,NaN
4,88061,tt0055459,"So Evil, So Young",NaN,[Drama],en,[en],1963-01-01,77.0,NaN,[],[],0.001662,NaN,NaN,"[prison, women's prison]","[Jill Ireland, Ellen Pollock, Joan Haythorne, ...",Godfrey Grayson,Mark Grantham,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43246,24428,tt0848228,The Avengers,The Avengers Collection,"[Science Fiction, Action, Adventure]",en,[en],2012-04-25,143.0,1.519558e+09,"[Paramount Pictures, Marvel Studios]",[United States of America],89.887648,12000.0,7.4,"[new york, shield, marvel comic, superhero, ba...","[Robert Downey Jr., Chris Evans, Mark Ruffalo,...",Joss Whedon,Joss Whedon,Stan Lee
43247,19995,tt0499549,Avatar,Avatar Collection,"[Action, Adventure, Fantasy, Science Fiction]",en,"[en, es]",2009-12-10,162.0,2.787965e+09,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",185.070892,12114.0,7.2,"[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron,James Cameron,James Cameron
43248,155,tt0468569,The Dark Knight,The Dark Knight Collection,"[Drama, Action, Crime, Thriller]",en,"[en, zh]",2008-07-16,152.0,1.004558e+09,"[DC Comics, Legendary Pictures, Warner Bros., ...","[United Kingdom, United States of America]",123.167259,12269.0,8.3,"[dc comics, crime fighter, secret identity, sc...","[Christian Bale, Michael Caine, Heath Ledger, ...",Christopher Nolan,Christopher Nolan,Charles Roven
43249,27205,tt1375666,Inception,NaN,"[Action, Thriller, Science Fiction, Mystery, A...",en,[en],2010-07-14,148.0,8.255328e+08,"[Legendary Pictures, Warner Bros., Syncopy]","[United Kingdom, United States of America]",29.108149,14075.0,8.1,"[loss of lover, dream, kidnapping, sleep, subc...","[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle...",Christopher Nolan,Christopher Nolan,Christopher Nolan


In [16]:
# clean up keywords 
keywords = movies_df.apply(lambda x: pd.Series(x["Keywords"]), axis = 1).stack().reset_index(level = 1, drop = True)
keywords = keywords.value_counts()
keywords = keywords[keywords >= 20]
keywords

# check the distribution and see if you increased the threshold how many keywords would you lose
# start with maybe seeing what keywords are present with a threshold of at least 50

woman director      2904
independent film    1825
murder              1201
based on novel       761
musical              681
                    ... 
perry mason           20
tour                  20
mumblecore            20
husband               20
forgiveness           20
Length: 1445, dtype: int64

In [17]:
# check distribution of keywords
#keywords.values
print(np.histogram(keywords.values))
from collections import Counter
keyword_dist = Counter(keywords.values)
keyword_dist

(array([1421,   16,    5,    0,    1,    0,    1,    0,    0,    1]), array([  20. ,  308.4,  596.8,  885.2, 1173.6, 1462. , 1750.4, 2038.8,
       2327.2, 2615.6, 2904. ]))


Counter({2904: 1,
         1825: 1,
         1201: 1,
         761: 1,
         681: 1,
         654: 1,
         620: 1,
         604: 1,
         592: 1,
         575: 1,
         536: 1,
         535: 1,
         534: 1,
         520: 1,
         438: 1,
         431: 1,
         426: 1,
         421: 1,
         399: 1,
         373: 1,
         351: 1,
         343: 1,
         326: 1,
         309: 1,
         300: 1,
         292: 1,
         286: 1,
         285: 1,
         284: 1,
         273: 1,
         269: 1,
         268: 1,
         264: 1,
         263: 1,
         259: 2,
         252: 1,
         251: 1,
         248: 1,
         246: 1,
         245: 1,
         243: 1,
         239: 1,
         236: 1,
         232: 1,
         228: 2,
         225: 1,
         224: 1,
         223: 1,
         218: 1,
         217: 1,
         216: 3,
         215: 1,
         214: 1,
         213: 1,
         206: 1,
         204: 1,
         201: 1,
         199: 2,
         19

In [18]:
def select_keywords(words):
    final_words = []
    for word in words:
        if word in keywords:
            final_words.append(word)
    return final_words

In [19]:
# only keep keywords that have frequency greater than 3
movies_df["Keywords"] = movies_df["Keywords"].apply(select_keywords).apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [20]:
# Collection - remove whitespace and convert to lowercase 
movies_df["Collection"] = movies_df["Collection"].str.lower().str.replace(" ", "")

# Cast - remove whitespace and convert to lowercase 
movies_df["Cast"] = movies_df["Cast"].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

# Director - remove whitespace and convert to lowercase 
movies_df["Director"] = movies_df["Director"].str.lower().str.replace(" ", "")

# Writer - remove whitespace and convert to lowercase 
movies_df["Writer"] = movies_df["Writer"].str.lower().str.replace(" ", "")

# Producer - remove whitespace and convert to lowercase 
movies_df["Producer"] = movies_df["Producer"].str.lower().str.replace(" ", "")

In [21]:
# replace NAN with empty space 
movies_df["Collection"] = movies_df["Collection"].fillna('')
movies_df["Director"] = movies_df["Director"].fillna('')
movies_df["Writer"] = movies_df["Writer"].fillna('')
movies_df["Producer"] = movies_df["Producer"].fillna('')

In [22]:
# experimenting with giving more weight to director -- if don't want comment this line
#movies_df["Director"] = movies_df["Director"].apply(lambda x: f"{x} {x} {x}")

# combine all string metadata
movies_df["Metadata"] = movies_df["Director"] + " " + movies_df["Writer"] + " " + movies_df["Producer"] + " " + movies_df["Collection"]

# combine list metadata
movies_df["Metadata2"] = movies_df["Genres"] + movies_df["Cast"] + movies_df["Keywords"]
movies_df["Metadata2"] = movies_df["Metadata2"].apply(lambda x: " ".join(x))

# merge all metadata to one column 
movies_df["Metadata"] = movies_df["Metadata"] + movies_df["Metadata2"]
# drop second metadata col
movies_df = movies_df.drop(columns=["Metadata2"])
movies_df

,ID,IMDB ID,Title,Collection,Genres,Language,Spoken Languages,Release Date,Runtime,Revenue,...,Production Countries,Popularity Rating,Vote Count,Vote Average,Keywords,Cast,Director,Writer,Producer,Metadata
0,461257,tt6980792,Queerama,,[],en,[en],2017-06-09,75.0,NaN,...,[United Kingdom],0.163015,NaN,NaN,[],[],daisyasquith,,,daisyasquith
1,92323,tt0081758,Willie and Phil,,[],en,[],1980-08-15,115.0,NaN,...,[],0.326500,NaN,NaN,[],"[michaelontkean, raysharkey, margotkidder]",paulmazursky,paulmazursky,,paulmazursky paulmazursky michaelontkean rays...
2,114838,tt0029949,Brother Rat,,[Comedy],en,[en],1938-10-29,87.0,NaN,...,[United States of America],0.174691,NaN,NaN,[basedonplayormusical],"[ronaldreagan, janewyman, priscillalane, wayne...",williamkeighley,jerrywald,,williamkeighley jerrywald Comedy ronaldreagan...
3,264723,tt0070580,Le pélican,,[],en,[],1974-02-06,83.0,NaN,...,[],0.000115,NaN,NaN,[],[],gérardblain,,,gérardblain
4,88061,tt0055459,"So Evil, So Young",,[Drama],en,[en],1963-01-01,77.0,NaN,...,[],0.001662,NaN,NaN,"[prison, women'sprison]","[jillireland, ellenpollock, joanhaythorne, oli...",godfreygrayson,markgrantham,,godfreygrayson markgrantham Drama jillireland...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43246,24428,tt0848228,The Avengers,theavengerscollection,"[Science Fiction, Action, Adventure]",en,[en],2012-04-25,143.0,1.519558e+09,...,[United States of America],89.887648,12000.0,7.4,"[newyork, marvelcomic, superhero, basedoncomic...","[robertdowneyjr., chrisevans, markruffalo, chr...",josswhedon,josswhedon,stanlee,josswhedon josswhedon stanlee theavengerscolle...
43247,19995,tt0499549,Avatar,avatarcollection,"[Action, Adventure, Fantasy, Science Fiction]",en,"[en, es]",2009-12-10,162.0,2.787965e+09,...,"[United States of America, United Kingdom]",185.070892,12114.0,7.2,"[cultureclash, future, society, spacetravel, f...","[samworthington, zoesaldana, sigourneyweaver, ...",jamescameron,jamescameron,jamescameron,jamescameron jamescameron jamescameron avatarc...
43248,155,tt0468569,The Dark Knight,thedarkknightcollection,"[Drama, Action, Crime, Thriller]",en,"[en, zh]",2008-07-16,152.0,1.004558e+09,...,"[United Kingdom, United States of America]",123.167259,12269.0,8.3,"[dccomics, crimefighter, secretidentity, sadis...","[christianbale, michaelcaine, heathledger, aar...",christophernolan,christophernolan,charlesroven,christophernolan christophernolan charlesroven...
43249,27205,tt1375666,Inception,,"[Action, Thriller, Science Fiction, Mystery, A...",en,[en],2010-07-14,148.0,8.255328e+08,...,"[United Kingdom, United States of America]",29.108149,14075.0,8.1,"[lossoflover, dream, kidnapping, heist, redemp...","[leonardodicaprio, josephgordon-levitt, ellenp...",christophernolan,christophernolan,christophernolan,christophernolan christophernolan christophern...


In [23]:
duplicate = movies_df[movies_df.duplicated(subset=["Title"])] 
duplicate

,ID,IMDB ID,Title,Collection,Genres,Language,Spoken Languages,Release Date,Runtime,Revenue,...,Production Countries,Popularity Rating,Vote Count,Vote Average,Keywords,Cast,Director,Writer,Producer,Metadata
585,187156,tt1515157,Jean-Luc Cinema Godard,,[Documentary],en,"[fr, en]",2009-09-30,24.0,NaN,...,[United Kingdom],0.004841,NaN,NaN,[],[],shaneo'sullivan,shaneo'sullivan,,shaneo'sullivan shaneo'sullivan Documentary
1354,99080,tt0022537,The Viking,,"[Action, Drama, Romance, Adventure]",en,[en],1931-06-21,70.0,NaN,...,[],0.002362,NaN,NaN,[],"[charlesstarrett, louisehuntington, arthurvint...",varickfrissell,,,varickfrissell Action Drama Romance Adventur...
1355,99080,tt0022537,The Viking,,"[Action, Drama, Romance, Adventure]",en,[en],1931-06-21,70.0,NaN,...,[],0.002362,NaN,NaN,[],"[charlesstarrett, louisehuntington, arthurvint...",georgemelford,,,georgemelford Action Drama Romance Adventure...
1356,99080,tt0022537,The Viking,,"[Action, Drama, Romance, Adventure]",en,[en],1931-06-21,70.0,NaN,...,[],0.002362,NaN,NaN,[],"[charlesstarrett, louisehuntington, arthurvint...",varickfrissell,,,varickfrissell Action Drama Romance Adventur...
2203,314283,tt4276834,A Perfect Christmas List,,"[Comedy, Drama, Family]",en,"[en, fr]",2014-12-14,86.0,NaN,...,[United States of America],0.605492,NaN,NaN,[christmas],"[ellenhollman, bethbroderick, aaronhill, richa...",fredolenray,petersullivan,,fredolenray petersullivan Comedy Drama Family...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42595,140300,tt2267968,Kung Fu Panda 3,kungfupandacollection,"[Action, Adventure, Animation, Comedy, Family]",en,[en],2016-01-23,95.0,5.211708e+08,...,"[China, United States of America]",14.696548,1630.0,6.7,"[china, martialarts, kungfu, village, sequel, ...","[jackblack, bryancranston, dustinhoffman, ange...",jenniferyuhnelson,glennberger,melissacobb,jenniferyuhnelson glennberger melissacobb kung...
42605,267860,tt3300542,London Has Fallen,...hasfallencollection,"[Action, Crime, Thriller]",en,[en],2016-03-02,99.0,2.057544e+08,...,"[Bulgaria, United States of America, United Ki...",15.037993,1656.0,5.8,"[londonengland, terrorist]","[gerardbutler, aaroneckhart, morganfreeman, an...",babaknajafi,creightonrothenberger,gerardbutler,babaknajafi creightonrothenberger gerardbutler...
42878,333371,tt1179933,10 Cloverfield Lane,,"[Thriller, Science Fiction, Drama]",en,[en],2016-03-10,103.0,1.082864e+08,...,[United States of America],14.421528,2537.0,6.8,"[kidnapping, paranoia, basement, apocalypse, c...","[maryelizabethwinstead, johngoodman, johngalla...",dantrachtenberg,damienchazelle,j.j.abrams,dantrachtenberg damienchazelle j.j.abrams Thri...
42906,296096,tt2674426,Me Before You,,"[Drama, Romance]",en,[en],2016-06-02,110.0,2.079451e+08,...,[United States of America],34.347590,2674.0,7.6,"[england, basedonnovel, depression, smalltown,...","[emiliaclarke, samclaflin, janetmcteer, charle...",theasharrock,scottneustadter,karenrosenfelt,theasharrock scottneustadter karenrosenfelt Dr...


In [24]:
# drop duplicates 
movies_df = movies_df.drop_duplicates(subset=["Title"])
movies_df

,ID,IMDB ID,Title,Collection,Genres,Language,Spoken Languages,Release Date,Runtime,Revenue,...,Production Countries,Popularity Rating,Vote Count,Vote Average,Keywords,Cast,Director,Writer,Producer,Metadata
0,461257,tt6980792,Queerama,,[],en,[en],2017-06-09,75.0,NaN,...,[United Kingdom],0.163015,NaN,NaN,[],[],daisyasquith,,,daisyasquith
1,92323,tt0081758,Willie and Phil,,[],en,[],1980-08-15,115.0,NaN,...,[],0.326500,NaN,NaN,[],"[michaelontkean, raysharkey, margotkidder]",paulmazursky,paulmazursky,,paulmazursky paulmazursky michaelontkean rays...
2,114838,tt0029949,Brother Rat,,[Comedy],en,[en],1938-10-29,87.0,NaN,...,[United States of America],0.174691,NaN,NaN,[basedonplayormusical],"[ronaldreagan, janewyman, priscillalane, wayne...",williamkeighley,jerrywald,,williamkeighley jerrywald Comedy ronaldreagan...
3,264723,tt0070580,Le pélican,,[],en,[],1974-02-06,83.0,NaN,...,[],0.000115,NaN,NaN,[],[],gérardblain,,,gérardblain
4,88061,tt0055459,"So Evil, So Young",,[Drama],en,[en],1963-01-01,77.0,NaN,...,[],0.001662,NaN,NaN,"[prison, women'sprison]","[jillireland, ellenpollock, joanhaythorne, oli...",godfreygrayson,markgrantham,,godfreygrayson markgrantham Drama jillireland...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43246,24428,tt0848228,The Avengers,theavengerscollection,"[Science Fiction, Action, Adventure]",en,[en],2012-04-25,143.0,1.519558e+09,...,[United States of America],89.887648,12000.0,7.4,"[newyork, marvelcomic, superhero, basedoncomic...","[robertdowneyjr., chrisevans, markruffalo, chr...",josswhedon,josswhedon,stanlee,josswhedon josswhedon stanlee theavengerscolle...
43247,19995,tt0499549,Avatar,avatarcollection,"[Action, Adventure, Fantasy, Science Fiction]",en,"[en, es]",2009-12-10,162.0,2.787965e+09,...,"[United States of America, United Kingdom]",185.070892,12114.0,7.2,"[cultureclash, future, society, spacetravel, f...","[samworthington, zoesaldana, sigourneyweaver, ...",jamescameron,jamescameron,jamescameron,jamescameron jamescameron jamescameron avatarc...
43248,155,tt0468569,The Dark Knight,thedarkknightcollection,"[Drama, Action, Crime, Thriller]",en,"[en, zh]",2008-07-16,152.0,1.004558e+09,...,"[United Kingdom, United States of America]",123.167259,12269.0,8.3,"[dccomics, crimefighter, secretidentity, sadis...","[christianbale, michaelcaine, heathledger, aar...",christophernolan,christophernolan,charlesroven,christophernolan christophernolan charlesroven...
43249,27205,tt1375666,Inception,,"[Action, Thriller, Science Fiction, Mystery, A...",en,[en],2010-07-14,148.0,8.255328e+08,...,"[United Kingdom, United States of America]",29.108149,14075.0,8.1,"[lossoflover, dream, kidnapping, heist, redemp...","[leonardodicaprio, josephgordon-levitt, ellenp...",christophernolan,christophernolan,christophernolan,christophernolan christophernolan christophern...


In [19]:
# save cleaned metadata to csv file 
outfile = "movies_metadata_cleaned.csv"
movies_df.to_csv(outfile, index=False)

## **Data Wrangling for Hybrid Recommender System**

Hybrid Recommender combining content based model and collaborative filtering model. Going to input the User ID and title of the movie and return top 20 movies that are similar based on both the metadata of the input movie and the user's preferences and predicted ratings of those movies.

In [27]:
movies_df

,index,ID,IMDB ID,Title,Collection,Genres,Language,Spoken Languages,Release Date,Runtime,...,Production Countries,Popularity Rating,Vote Count,Vote Average,Keywords,Cast,Director,Writer,Producer,Metadata
0,0,461257,tt6980792,Queerama,NaN,[],en,['en'],2017-06-09,75.0,...,['United Kingdom'],0.163015,NaN,NaN,[],[],daisyasquith,NaN,NaN,daisyasquith
1,1,92323,tt0081758,Willie and Phil,NaN,[],en,[],1980-08-15,115.0,...,[],0.326500,NaN,NaN,[],"['michaelontkean', 'raysharkey', 'margotkidder']",paulmazursky,paulmazursky,NaN,paulmazursky paulmazursky michaelontkean rays...
2,2,114838,tt0029949,Brother Rat,NaN,['Comedy'],en,['en'],1938-10-29,87.0,...,['United States of America'],0.174691,NaN,NaN,['basedonplayormusical'],"['ronaldreagan', 'janewyman', 'priscillalane',...",williamkeighley,jerrywald,NaN,williamkeighley jerrywald Comedy ronaldreagan...
3,3,264723,tt0070580,Le pélican,NaN,[],en,[],1974-02-06,83.0,...,[],0.000115,NaN,NaN,[],[],gérardblain,NaN,NaN,gérardblain
4,4,88061,tt0055459,"So Evil, So Young",NaN,['Drama'],en,['en'],1963-01-01,77.0,...,[],0.001662,NaN,NaN,"['prison', ""women'sprison""]","['jillireland', 'ellenpollock', 'joanhaythorne...",godfreygrayson,markgrantham,NaN,godfreygrayson markgrantham Drama jillireland...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42273,42273,24428,tt0848228,The Avengers,theavengerscollection,"['Science Fiction', 'Action', 'Adventure']",en,['en'],2012-04-25,143.0,...,['United States of America'],89.887648,12000.0,7.4,"['newyork', 'shield', 'marvelcomic', 'superher...","['robertdowneyjr.', 'chrisevans', 'markruffalo...",josswhedon,josswhedon,stanlee,josswhedon josswhedon stanlee theavengerscolle...
42274,42274,19995,tt0499549,Avatar,avatarcollection,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",en,"['en', 'es']",2009-12-10,162.0,...,"['United States of America', 'United Kingdom']",185.070892,12114.0,7.2,"['cultureclash', 'future', 'spacewar', 'spacec...","['samworthington', 'zoesaldana', 'sigourneywea...",jamescameron,jamescameron,jamescameron,jamescameron jamescameron jamescameron avatarc...
42275,42275,155,tt0468569,The Dark Knight,thedarkknightcollection,"['Drama', 'Action', 'Crime', 'Thriller']",en,"['en', 'zh']",2008-07-16,152.0,...,"['United Kingdom', 'United States of America']",123.167259,12269.0,8.3,"['dccomics', 'crimefighter', 'secretidentity',...","['christianbale', 'michaelcaine', 'heathledger...",christophernolan,christophernolan,charlesroven,christophernolan christophernolan charlesroven...
42276,42276,27205,tt1375666,Inception,NaN,"['Action', 'Thriller', 'Science Fiction', 'Mys...",en,['en'],2010-07-14,148.0,...,"['United Kingdom', 'United States of America']",29.108149,14075.0,8.1,"['lossoflover', 'dream', 'kidnapping', 'sleep'...","['leonardodicaprio', 'josephgordon-levitt', 'e...",christophernolan,christophernolan,christophernolan,christophernolan christophernolan christophern...


In [69]:
movies_ID_map = pd.read_csv("Movies Data/links.csv")[['movieId', 'tmdbId']]
movies_ID_map.columns = ['Movie ID', 'ID']

# drop movies with no ID
movies_ID_map = movies_ID_map.dropna(subset=["ID"])
movies_ID_map["ID"] = movies_ID_map["ID"].astype(int)

# drop duplicates 
movies_ID_map = movies_ID_map.drop_duplicates(subset=["ID"])
movies_ID_map

,Movie ID,ID
0,1,862
1,2,8844
2,3,15602
3,4,31357
4,5,11862
...,...,...
45838,176269,439050
45839,176271,111109
45840,176273,67758
45841,176275,227506


In [70]:
movies_titles = movies_df[["Title", "ID"]]

# drop any null values
movies_titles = movies_titles.dropna()
movies_titles["ID"] = movies_titles["ID"].astype(str).astype(int)

# merge to create a map between the movie ID and tmdb ID and movie title
#movies_titles = movies_titles.merge(movie_ID_map, how="left", on="ID")
movies_ID_map = movies_titles.merge(movies_ID_map, how="left", on="ID")
movies_ID_map

,Title,ID,Movie ID
0,Queerama,461257,176279
1,Willie and Phil,92323,112577
2,Brother Rat,114838,112548
3,Le pélican,264723,112510
4,"So Evil, So Young",88061,112467
...,...,...,...
42272,Deadpool,293660,122904
42273,The Avengers,24428,89745
42274,Avatar,19995,72998
42275,The Dark Knight,155,58559


In [76]:
# save movies ID map to csv file 
outfile = "movies_ID_map.csv"
movies_ID_map.to_csv(outfile, index=False)